# Code for Cahn-Hilliard phase separation without mechanical coupling.

## 2D phase separation study.


Degrees of freedom: 
  - scalar chemical potential: we use normalized  mu = mu/RT 
  - species concentration:  we use normalized  c= Omega*cmat 
  
### Units:
- Length: um
- Mass: kg
- Time: s
- Amount of substance: pmol
- Temperature: K
- Mass density: kg/um^3
- Force: uN
- Stress: MPa
- Energy: pJ
- Species concentration: pmol/um^3
- Chemical potential: pJ/pmol
- Molar volume: um^3/pmol
- Species diffusivity: um^2/s
- Boltzmann Constant: 1.38E-11 pJ/K
- Gas constant: 8.314  pJ/(pmol K)

### By
  Eric Stewart      and      Lallit Anand
ericstew@mit.edu            anand@mit.edu

October 2023

Modified for FenicsX by Jorge Nin
jorgenin@mit.edu

In [1]:
import numpy as np


from mpi4py import MPI
from petsc4py import PETSc

from dolfinx import fem, mesh, io, plot, log
from dolfinx.fem import Constant, dirichletbc, Function, FunctionSpace, Expression
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from dolfinx.io import VTXWriter
import ufl
from ufl import (
    TestFunction,
    TrialFunction,
    Identity,
    grad,
    det,
    div,
    dev,
    inv,
    tr,
    sqrt,
    conditional,
    gt,
    dx,
    inner,
    derivative,
    dot,
    ln,
    split,
    tanh,
    as_tensor,
    as_vector,
    ge
)
from datetime import datetime
from dolfinx.plot import vtk_mesh
from hilliard_models import Cahn_Hillard_3D_no_mech
import pyvista

pyvista.set_jupyter_backend("client")
## Define temporal parameters
import random

# DEFINE GEOMETRY

In [2]:
problemName = "Canh Hillard 3D"

# Square edge length
L0 = 0.8  # 800 nm box, after Di Leo et al. (2014)

# Number of elements along each side
N = 50

# Create square mesh
domain = mesh.create_box(MPI.COMM_WORLD, [(0, 0,0), (L0, L0,L0)], [N, N,N],cell_type=mesh.CellType.hexahedron)

## Visualize the geometry

In [3]:
#sysctl -a|grep "\.shm"

plotter = pyvista.Plotter()
vtkdata = vtk_mesh(domain, domain.topology.dim)
grid = pyvista.UnstructuredGrid(*vtkdata)
actor = plotter.add_mesh(grid, show_edges=True)
plotter.show()
plotter.close()

Widget(value="<iframe src='http://localhost:60107/index.html?ui=P_0x29adb7c10_0&reconnect=auto' style='width: …

# Simulation Time Control

In [4]:
t = 0.0  # initialization of time
Ttot = 4  # total simulation time
dt = 0.01  # Initial time step size, here we will use adaptive time-stepping

In [5]:
dk = Constant(domain,dt)

hillard_problem = Cahn_Hillard_3D_no_mech(domain)
hillard_problem.Kinematics()
hillard_problem.WeakForms(dk)

# Setup Output Files

In [6]:
U1 = ufl.VectorElement("Lagrange", domain.ufl_cell(), 1)
V2 = fem.FunctionSpace(domain, U1)#Vector function space
V1 = fem.FunctionSpace(domain, hillard_problem.P1)#Scalar function space

mu_vis = Function(V1)
mu_vis.name = "mu"
mu_expr = Expression(hillard_problem.mu,V1.element.interpolation_points())

c_vis = Function(V1)
c_vis.name = "c"
c_expr = Expression(hillard_problem.c,V1.element.interpolation_points())

vtk = VTXWriter(domain.comm,"results/"+problemName+".bp", [mu_vis,c_vis], engine="BP4" )

def interp_and_save(t, file):
   
    mu_vis.interpolate(mu_expr)
    c_vis.interpolate(c_expr)

    file.write(t)


# Boundary Conditions

In [7]:
# Nothing! Just let the system evolve on its own.
bcs = []

# SETUP NONLINEAR PROBLEM

In [8]:
import os
step = "Swell"
jit_options ={"cffi_extra_compile_args":["-O3","-ffast-math"]}

problem = NonlinearProblem(hillard_problem.Res, hillard_problem.w, bcs, hillard_problem.a)


solver = NewtonSolver(MPI.COMM_WORLD, problem)
solver.convergence_criterion = "incremental"
solver.rtol = 1e-8
solver.atol = 1e-8
solver.max_it = 50
solver.report = True


ksp = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_max_it"] = 30
opts[f"{option_prefix}ksp_type"] = "gmres"
opts[f"{option_prefix}pc_type"] = "ksp"
ksp.setFromOptions()

startTime = datetime.now()
print("------------------------------------")
print("Simulation Start")
print("------------------------------------")

step = "Evolve"

#if os.path.exists("results/"+problemName+".bp"):
#    os.remove("results/"+problemName+".xdmf")
#    os.remove("results/"+problemName+".h5")

#vtk.write_mesh(domain)
t = 0.0

interp_and_save(t, vtk)
ii = 0
bisection_count = 0
lasttimestep = hillard_problem.w_old_2.x.array.copy()
while t < Ttot:
    # increment time
    t += float(dk) 
    # increment counter
    ii +=1
    

    # Solve the problem
    try:
        (iter, converged) = solver.solve(hillard_problem.w)
        
        hillard_problem.w.x.scatter_forward()
        
        
        lasttimestep[:] = hillard_problem.w_old_2.x.array
        hillard_problem.w_old_2.x.array[:] = hillard_problem.w_old.x.array
        hillard_problem.w_old.x.array[:] = hillard_problem.w.x.array
        
        interp_and_save(t, vtk)
        if ii % 1 == 0:
            now = datetime.now()
            current_time = now.strftime("%H:%M:%S")
            print("Step: {} |   Increment: {} | Iterations: {}".format(step, ii, iter))
            print("Simulation Time: {} s | dt: {} s".format(round(t, 2), round(dt, 3)))
            print()
        
        if iter <= 2:
            dt = 1.5 * dt
            dk.value = dt
        # If the newton solver takes 5 or more iterations,
        # decrease the time step by a factor of 2:
        elif iter >= 5:
            dt = dt / 2
            dk.value =dt

        #Reset Biseciton Counter
        bisection_count = 0
        
    except: # Break the loop if solver fails
        bisection_count += 1
        
        if bisection_count > 5:
            print("Error: Too many bisections")
            break
        
        print( "Error Halfing Time Step")
        t = t - float(dk)
        dt = dt / 2
        dk.value = dt
        print(f"New Time Step: {dt}")
        
        hillard_problem.w.x.array[:] = hillard_problem.w_old.x.array
        hillard_problem.w_old.x.array[:] = hillard_problem.w_old_2.x.array
        hillard_problem.w_old_2.x.array[:] = lasttimestep
        

#End Analysis
vtk.close()
endTime = datetime.now()
print("------------------------------------")
print("Simulation End")
print("------------------------------------")
print("Total Time: {}".format(endTime - startTime))
print("------------------------------------")



    
    

------------------------------------
Simulation Start
------------------------------------
Step: Evolve |   Increment: 1 | Iterations: 4
Simulation Time: 0.01 s | dt: 0.01 s

Step: Evolve |   Increment: 2 | Iterations: 3
Simulation Time: 0.02 s | dt: 0.01 s

Step: Evolve |   Increment: 3 | Iterations: 3
Simulation Time: 0.03 s | dt: 0.01 s

Step: Evolve |   Increment: 4 | Iterations: 3
Simulation Time: 0.04 s | dt: 0.01 s

Step: Evolve |   Increment: 5 | Iterations: 3
Simulation Time: 0.05 s | dt: 0.01 s

Step: Evolve |   Increment: 6 | Iterations: 3
Simulation Time: 0.06 s | dt: 0.01 s

Step: Evolve |   Increment: 7 | Iterations: 3
Simulation Time: 0.07 s | dt: 0.01 s

Step: Evolve |   Increment: 8 | Iterations: 3
Simulation Time: 0.08 s | dt: 0.01 s

Step: Evolve |   Increment: 9 | Iterations: 3
Simulation Time: 0.09 s | dt: 0.01 s

Step: Evolve |   Increment: 10 | Iterations: 3
Simulation Time: 0.1 s | dt: 0.01 s

Step: Evolve |   Increment: 11 | Iterations: 3
Simulation Time: 0.11 

ValueError: cannot reshape array of size 1665674468 into shape (265302,265302)

array(<petsc4py.PETSc.Mat object at 0x2a9ae55d0>, dtype=object)